In [1]:
import time
from Crypto.PublicKey import ECC
from Crypto.Hash import SHA256
from Crypto.Signature import DSS

def bench(fn, n=2000):
    t0 = time.perf_counter()
    for _ in range(n):
        fn()
    t1 = time.perf_counter()
    return (t1 - t0) / n

def us(x):
    return x * 1e6

def flip_last_byte(b):
    if not b:
        return b
    x = bytearray(b)
    x[-1] ^= 1
    return bytes(x)

def flip_last_bit_bytes(b):
    if not b:
        return b
    x = bytearray(b)
    x[-1] ^= 1
    return bytes(x)

messages = [
    b"Hi",
    b"MRKM Lab 3 - ECDSA control example. This is a medium length message.",
    (b"A" * 4096),
]

curves = ["P-256", "P-384"]
mode = "fips-186-3"
n_bench = 2000


In [2]:
print("mode =", mode)
print("hash =", "SHA-256")
print("n_bench =", n_bench)
print("messages_bytes =", [len(m) for m in messages])


mode = fips-186-3
hash = SHA-256
n_bench = 2000
messages_bytes = [2, 68, 4096]


In [3]:
for curve in curves:
    key = ECC.generate(curve=curve)
    pub = key.public_key()

    pub_der = pub.export_key(format="DER")
    priv_der = key.export_key(format="DER")

    signer = DSS.new(key, mode)
    verifier = DSS.new(pub, mode)

    print("\n=== curve =", curve, "===")
    print("public_key_der_bytes =", len(pub_der))
    print("private_key_der_bytes =", len(priv_der))

    for i, msg in enumerate(messages, start=1):
        h = SHA256.new(msg)
        sig = signer.sign(h)

        ok = True
        try:
            verifier.verify(h, sig)
        except ValueError:
            ok = False

        bad_msg = flip_last_byte(msg)
        bad_ok = True
        try:
            verifier.verify(SHA256.new(bad_msg), sig)
        except ValueError:
            bad_ok = False

        bad_sig = flip_last_bit_bytes(sig)
        bad_sig_ok = True
        try:
            verifier.verify(h, bad_sig)
        except ValueError:
            bad_sig_ok = False

        def sign_once():
            return signer.sign(SHA256.new(msg))

        def verify_once():
            verifier.verify(SHA256.new(msg), sig)

        t_sign = bench(sign_once, n=n_bench)
        t_verify = bench(verify_once, n=n_bench)

        print("\nmessage", i, "len_bytes =", len(msg))
        print("demo_verify_ok =", ok)
        print("negative_test_modified_message_ok =", bad_ok)
        print("negative_test_modified_signature_ok =", bad_sig_ok)
        print("signature_bytes =", len(sig))
        print("avg_sign_us =", us(t_sign))
        print("avg_verify_us =", us(t_verify))



=== curve = P-256 ===
public_key_der_bytes = 91
private_key_der_bytes = 138

message 1 len_bytes = 2
demo_verify_ok = True
negative_test_modified_message_ok = False
negative_test_modified_signature_ok = False
signature_bytes = 64
avg_sign_us = 280.30850004870445
avg_verify_us = 923.142000043299

message 2 len_bytes = 68
demo_verify_ok = True
negative_test_modified_message_ok = False
negative_test_modified_signature_ok = False
signature_bytes = 64
avg_sign_us = 299.15094998432323
avg_verify_us = 933.2967500085942

message 3 len_bytes = 4096
demo_verify_ok = True
negative_test_modified_message_ok = False
negative_test_modified_signature_ok = False
signature_bytes = 64
avg_sign_us = 313.20184998912737
avg_verify_us = 974.313999991864

=== curve = P-384 ===
public_key_der_bytes = 120
private_key_der_bytes = 185

message 1 len_bytes = 2
demo_verify_ok = True
negative_test_modified_message_ok = False
negative_test_modified_signature_ok = False
signature_bytes = 96
avg_sign_us = 670.41615000